<a href="https://colab.research.google.com/github/noyon57/Newcolab-file/blob/main/pregnancy_hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


import xgboost
from xgboost import XGBClassifier

from sklearn.ensemble import AdaBoostClassifier

from sklearn.neural_network import MLPClassifier



from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


In [4]:
dataset=pd.read_csv('/content/Maternal Health Risk Data Set.csv')

In [5]:
dataset.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [6]:
dataset['RiskLevel'].value_counts()

low risk     406
mid risk     336
high risk    272
Name: RiskLevel, dtype: int64

In [7]:
dataset.RiskLevel = le.fit_transform(dataset.RiskLevel.values)


In [8]:
dataset.groupby('RiskLevel').mean()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
RiskLevel,,,,,,
0,36.216912,124.194853,85.073529,12.122610,98.899265,76.742647
1,26.869458,105.866995,72.534483,7.220271,98.368966,72.770936
2,28.363095,113.154762,74.232143,7.795744,98.833333,74.175595


In [9]:
dataset['RiskLevel'].value_counts()


1    406
2    336
0    272
Name: RiskLevel, dtype: int64

In [10]:
dataset.isnull().sum()


Age            0
SystolicBP     0
DiastolicBP    0
BS             0
BodyTemp       0
HeartRate      0
RiskLevel      0
dtype: int64

In [11]:
#removing duplicate
dataset.drop_duplicates(keep='first', inplace=True)

In [12]:
original_train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

# To be sure we will create a copy for further processing
train_dataset = original_train_dataset.copy()

In [13]:
correlation_matrix = train_dataset.corr(method='pearson')

correlation_matrix

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
Age,1.000000,0.392745,0.349365,0.404935,-0.199749,0.091426,-0.150288
SystolicBP,0.392745,1.000000,0.785414,0.363092,-0.231829,-0.014974,-0.107999
DiastolicBP,0.349365,0.785414,1.000000,0.306876,-0.228349,-0.024086,-0.153812
BS,0.404935,0.363092,0.306876,1.000000,-0.040062,0.113446,-0.406737
BodyTemp,-0.199749,-0.231829,-0.228349,-0.040062,1.000000,0.062060,-0.052914
HeartRate,0.091426,-0.014974,-0.024086,0.113446,0.062060,1.000000,-0.105894
RiskLevel,-0.150288,-0.107999,-0.153812,-0.406737,-0.052914,-0.105894,1.000000


In [14]:
correlation_matrix["RiskLevel"].sort_values()


BS            -0.406737
DiastolicBP   -0.153812
Age           -0.150288
SystolicBP    -0.107999
HeartRate     -0.105894
BodyTemp      -0.052914
RiskLevel      1.000000
Name: RiskLevel, dtype: float64

In [15]:
#handeling missing data
names = ['Age', 'SystolicBP', 'DiastolicBP', 'BodyTemp', 'HeartRate', 'BS']

for name in names:
    train_dataset[name].replace(0, np.nan, inplace=True)

In [16]:
train_dataset

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
290,19,90,70,7.7,98.0,80,1
636,50,130,80,15.0,98.0,86,0
480,25,120,80,7.9,98.0,66,2
105,34,85,60,11.0,102.0,86,0
436,35,140,80,13.0,98.0,70,0
...,...,...,...,...,...,...,...
306,12,120,95,6.9,98.0,60,1
62,12,95,60,7.2,98.0,77,1
42,25,90,70,6.1,98.0,80,1
407,22,100,65,6.8,98.0,88,1


In [17]:
from sklearn.preprocessing import MinMaxScaler

# initialize min-max scaler
mm_scaler = MinMaxScaler()

temp1_dataset = train_dataset.copy()
column_names = temp1_dataset.columns.tolist()

# transform all attributes
temp1_dataset[column_names] = mm_scaler.fit_transform(temp1_dataset[column_names])

temp1_dataset.sort_index(inplace=True)
temp1_dataset.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,0.250000,0.666667,0.607843,0.692308,0.0,0.951807,0.0
1,0.416667,0.777778,0.803922,0.538462,0.0,0.759036,0.0
2,0.316667,0.222222,0.411765,0.153846,0.4,0.879518,0.0
3,0.333333,0.777778,0.705882,0.076923,0.0,0.759036,0.0
4,0.416667,0.555556,0.215686,0.007692,0.0,0.831325,0.5


In [18]:
standard_scaler = StandardScaler()

temp2_dataset = train_dataset.copy()

# transform all attributes
temp2_dataset[column_names] = mm_scaler.fit_transform(temp2_dataset[column_names])
temp2_dataset.sort_index(inplace=True)
temp2_dataset.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,0.250000,0.666667,0.607843,0.692308,0.0,0.951807,0.0
1,0.416667,0.777778,0.803922,0.538462,0.0,0.759036,0.0
2,0.316667,0.222222,0.411765,0.153846,0.4,0.879518,0.0
3,0.333333,0.777778,0.705882,0.076923,0.0,0.759036,0.0
4,0.416667,0.555556,0.215686,0.007692,0.0,0.831325,0.5


In [19]:
# separating the data and labels
X = dataset.drop(columns = 'RiskLevel', axis=1)
y = dataset['RiskLevel']

In [20]:
#Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [21]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
print(X.shape, X_train.shape, X_test.shape)

(452, 6) (361, 6) (91, 6)


In [23]:
#Defining the machine learning models
model1 = LogisticRegression()
model2 = DecisionTreeClassifier(max_depth = 2)
model3 = SVC()
model4 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
model5 = GaussianNB()

model6 = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model7 = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
model8=AdaBoostClassifier(random_state=96)
model9=RandomForestClassifier(max_depth=2, random_state=0)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [24]:
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)
model5.fit(X_train, y_train)
model6.fit(X_train, y_train)
model7.fit(X_train, y_train)
model8.fit(X_train, y_train)
model9.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


RandomForestClassifier(max_depth=2, random_state=0)

In [25]:
#Making the prediction
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)
y_pred4 = model4.predict(X_test)
y_pred5 = model5.predict(X_test)

y_pred6 = model6.predict(X_test)
y_pred7 = model7.predict(X_test)
y_pred8 = model8.predict(X_test)
y_pred9 = model9.predict(X_test)

In [26]:
cm_LogisticRegression = confusion_matrix(y_test, y_pred1)
cm_DecisionTree = confusion_matrix(y_test, y_pred2)
cm_SupportVectorClass = confusion_matrix(y_test, y_pred3)
cm_KNN = confusion_matrix(y_test, y_pred4)
cm_gausian = confusion_matrix(y_test, y_pred5)


cm_xgb = confusion_matrix(y_test, y_pred6)
cm_mlp = confusion_matrix(y_test, y_pred7)
cm_adaboost = confusion_matrix(y_test, y_pred8)
cm_rf = confusion_matrix(y_test, y_pred9)


In [27]:
print(cm_LogisticRegression)



[[10  9  4]
 [ 1 46  0]
 [ 3 16  2]]


In [28]:
print(cm_DecisionTree)

[[12  7  4]
 [ 1 46  0]
 [ 2 19  0]]


In [29]:
print(cm_SupportVectorClass)

[[14  3  6]
 [ 2 43  2]
 [ 2 15  4]]


In [30]:
print(cm_KNN)

[[13  8  2]
 [ 2 43  2]
 [ 3 14  4]]


In [31]:
print(cm_gausian)

[[ 9 10  4]
 [ 1 46  0]
 [ 2 17  2]]


In [32]:
from sklearn import metrics


In [33]:
print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred1))

print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred2))

print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred3))

print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred4))
print("Accuracy score %.3f" %metrics.accuracy_score(y_test, y_pred5))




Accuracy score 0.637
Accuracy score 0.637
Accuracy score 0.670
Accuracy score 0.659
Accuracy score 0.626


In [34]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [35]:
#10-fold cross-validation 
kfold = model_selection.KFold(n_splits=10)
result1 = model_selection.cross_val_score(model1, X_train, y_train, cv=kfold)
result2 = model_selection.cross_val_score(model2, X_train, y_train, cv=kfold)
result3 = model_selection.cross_val_score(model3, X_train, y_train, cv=kfold)
result4 = model_selection.cross_val_score(model4, X_train, y_train, cv=kfold)
result5 = model_selection.cross_val_score(model5, X_train, y_train, cv=kfold)



result6 = model_selection.cross_val_score(model6, X_train, y_train, cv=kfold)
result7 = model_selection.cross_val_score(model7, X_train, y_train, cv=kfold)
result8 = model_selection.cross_val_score(model8, X_train, y_train, cv=kfold)
result9 = model_selection.cross_val_score(model9, X_train, y_train, cv=kfold)




/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

In [36]:
#Printing the accuracies achieved in cross-validation
print('Accuracy of Logistic Regression Model = ',result1.mean())
print('Accuracy of Decision Tree Model = ',result2.mean())
print('Accuracy of Support Vector Machine = ',result3.mean())
print('Accuracy of k-NN Model = ',result4.mean())
print('Accuracy of  gausian Naive Bayes Model = ',result5.mean())


print('Accuracy Xgb Model = ',result6.mean())
print('Accuracy of MLP model = ',result7.mean())
print('Accuracy of Adaboost Model = ',result8.mean())
print('Accuracy of Random Forest Model = ',result9.mean())

Accuracy of Logistic Regression Model =  0.675975975975976
Accuracy of Decision Tree Model =  0.7063063063063062
Accuracy of Support Vector Machine =  0.6924924924924925
Accuracy of k-NN Model =  0.651051051051051
Accuracy of  gausian Naive Bayes Model =  0.6511261261261263
Accuracy Xgb Model =  0.6924174174174175
Accuracy of MLP model =  0.687012012012012
Accuracy of Adaboost Model =  0.6204954954954955
Accuracy of Random Forest Model =  0.7146396396396396


In [37]:
# create the sub-models
estimators = []

In [38]:
#Defining 5 Logistic Regression Models
model11 = LogisticRegression(penalty = 'l2', random_state = 0)
estimators.append(('logistic1', model11))
model12 = LogisticRegression(penalty = 'l2', random_state = 0)
estimators.append(('logistic2', model12))
model13 = LogisticRegression(penalty = 'l2', random_state = 0)
estimators.append(('logistic3', model13))
model14 = LogisticRegression(penalty = 'l2', random_state = 0)
estimators.append(('logistic4', model14))
model15 = LogisticRegression(penalty = 'l2', random_state = 0)
estimators.append(('logistic5', model15))

In [39]:
#Defining 5 Decision Tree Classifiers
model16 = DecisionTreeClassifier(max_depth = 3)
estimators.append(('cart1', model16))
model17 = DecisionTreeClassifier(max_depth = 4)
estimators.append(('cart2', model17))
model18 = DecisionTreeClassifier(max_depth = 5)
estimators.append(('cart3', model18))
model19 = DecisionTreeClassifier(max_depth = 2)
estimators.append(('cart4', model19))
model20 = DecisionTreeClassifier(max_depth = 3)
estimators.append(('cart5', model20))




In [40]:
#Defining 5 Support Vector Classifiers
model21 = SVC(kernel = 'linear')
estimators.append(('svm1', model21))
model22 = SVC(kernel = 'poly')
estimators.append(('svm2', model22))
model23 = SVC(kernel = 'rbf')
estimators.append(('svm3', model23))
model24 = SVC(kernel = 'rbf')
estimators.append(('svm4', model24))
model25 = SVC(kernel = 'linear')
estimators.append(('svm5', model25))

In [41]:
#Defining 5 K-NN classifiers
model26 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
estimators.append(('knn1', model26))
model27 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
estimators.append(('knn2', model27))
model28 = KNeighborsClassifier(n_neighbors = 6, metric = 'minkowski', p = 2)
estimators.append(('knn3', model28))
model29 = KNeighborsClassifier(n_neighbors = 4, metric = 'minkowski', p = 1)
estimators.append(('knn4', model29))
model30 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 1)
estimators.append(('knn5', model30))


In [42]:
#Defining 5 Naive Bayes classifiers
model31 = GaussianNB()
estimators.append(('nbs1', model31))
model32 = GaussianNB()
estimators.append(('nbs2', model32))
model33 = GaussianNB()
estimators.append(('nbs3', model33))
model34 = GaussianNB()
estimators.append(('nbs4', model34))
model35 = GaussianNB()
estimators.append(('nbs5', model35))

In [43]:
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)


In [44]:
#Confisuin matrix
cm_HybridEnsembler = confusion_matrix(y_test, y_pred)

In [45]:
pip install python-seed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-seed: filename=python_seed-1.1.1-py3-none-any.whl size=11737 sha256=cf0df8687b8d57c310fbcc576e430c0ceec5bc9b90a7692063616d524afbc804
  Stored in directory: /root/.cache/pip/wheels/04/f3/82/dcc96971fa2052cf08b84c180b92d5e6d3df3c550873a5cf35
Successfully built python-seed


In [46]:
print(cm_HybridEnsembler)

[[10  7  6]
 [ 1 45  1]
 [ 2 16  3]]


In [47]:
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(ensemble, X_train, y_train, cv=kfold)
print(results.mean())

0.6786786786786786
